# Defining a Web API in a Notebook

In this notebook, we'll define a RESTful web API for adding, updating, removing, and listing the members of a contact list. We'll then deploy our notebook as a service usign the Jupyter Kernel Gateway.

The contact list supported by this notebook is intentionally simple. See https://github.com/jupyter/kernel_gateway_demos for more complex examples and http://jupyter-kernel-gateway.readthedocs.io/en/latest/http-mode.html for full documentation.

**Table of Contents**

1. [Definition](#Definition)
2. [Implementation](#Implementation)
    1. [Create a contact](#Create-a-contact)
    2. [Update a contact](#Update-a-contact)
    3. [Delete a contact](#Delete-a-contact)
    4. [Get contacts](#Get-contacts)
3. [Deployment](#Deployment)
4. [Test](#Test)
5. [Improvement](#Improvement)

## Definition

Let's start by outlining the resources for our contact list.

* `POST /contacts` &rarr; create a new contact
* `PUT /contacts/:contact_id` &rarr; update a contact
* `DELETE /contacts/:contact_id` &rarr; delete a contact
* `GET /contacts?name=<regex>` &rarr; get contacts, optionally filtered by name

We'll also state that requests and responses should carry JSON content for consistency.

## Implementation

In [47]:
import json
import os
import uuid

For simplicity, we'll use a global dictionary to store our contact list. If we later want to make our list persistent or our web service scale to multiple workers, we can switch to a true data store.

In [53]:
contacts = {}

These are the fields we'll allow for each contact.

In [54]:
fields = ['name', 'phone', 'address']

### Create a contact

We'll want to get the values for the name, phone, and address fields from the client when we create a contact. The kernel gateway will set the `REQUEST` variable to a JSON string containing information from the client. Let's synthesize an example request here in order to develop the code to add a contact to our contact list.

In [75]:
REQUEST = json.dumps({
    'body': {
        'name': 'Jane Doe',
        'phone': '888-555-5245',
        'address': '123 Bellview Drive, Somewhere, NC'
    }
})

Now let's write the handler code. We'll also annotate it so that the kernel gateway knows the code in this cell should execute when a client sends a HTTP `POST` request to the `/contacts` path.

In [76]:
# POST /contacts
# decode the request
req = json.loads(REQUEST)
# pull out the body
body = req['body']
# generate a new contact ID
new_contact_id = str(uuid.uuid4())
# put what we can about the contact in the dictionary
contacts[new_contact_id] = {field: body.get(field) for field in fields}
print(json.dumps({'contact_id': new_contact_id}))

{"contact_id": "c50c57ca-a23f-4eb2-8254-76c86c032576"}


We can print the contacts to see if it contains the data from our sample request, and see that it does.

In [77]:
contacts

{'c50c57ca-a23f-4eb2-8254-76c86c032576': {'address': '123 Bellview Drive, Somewhere, NC',
  'name': 'Jane Doe',
  'phone': '888-555-5245'}}

It's worth pointing out that we can put development code like this in our notebook without harming how it works with the kernel gateway as long as our notebook can run top to bottom. With a little more effort, we can even write basic tests in our notebook that run only when we're authoring or editing the notebook.

### Update a contact

We can follow the same pattern to implement updates to existing contacts. We start with an example request.

In [78]:
REQUEST = json.dumps({
    'body': {
        'name': 'Jane and John Doe',
        'phone': '888-555-5245',
        'address': '321 Viewbell Lane, Somewhere Else, SC'
    },
    'path': {
        'contact_id': new_contact_id
    }
})

This time, we need to know the identity assigned to the contact we're updating. We'll get that by declaring a variable in the path of the resource, `contact_id`, and read the value from the request.

In [79]:
# PUT /contacts/:contact_id
req = json.loads(REQUEST)
body = req['body']
contact_id = req['path']['contact_id']
if contact_id in contacts:
    contacts[contact_id] = {field: body.get(field) for field in fields}
    status = 200
else:
    status = 404

In [80]:
contacts

{'c50c57ca-a23f-4eb2-8254-76c86c032576': {'address': '321 Viewbell Lane, Somewhere Else, SC',
  'name': 'Jane and John Doe',
  'phone': '888-555-5245'}}

In [81]:
# ResponseInfo PUT /contacts/:contact_id
print(json.dumps({
    "status" : status
}))

{"status": 200}


### Delete a contact

Our deletion code is much the same. We take a contact ID and remove it from our dict if it exists. We respond with a reasonable status code for successful deletion or a failure when the given ID is not found.

In [82]:
# DELETE /contacts/:contact_id
req = json.loads(REQUEST)
contact_id = req['path']['contact_id']
if contact_id in contacts:
    del contacts[contact_id]
    # HTTP status code for no body
    status = 204
else:
    # HTTP status code for not found
    status = 404

In [ ]:
# ResponseInfo DELETE /contacts/:contact_id
print(json.dumps({
    "status" : status
}))

### Get contacts

TODO: describe, implement

In [70]:
def filter_by_name(name_regex, contacts):
    # TODO: implement
    if name is not None:
        return {}
    else:
        return contacts
        

In [69]:
# GET /contacts
req = json.loads(REQUEST)
name_regex = req.get('query', {}).get('name')
hits = filter_by_name(name_regex, contacts)
print(json.dumps(hits))

{}


## Deployment

TODO: give command line

## Test

TODO: explain how to run against local KG and complete code

In [83]:
import requests

In [ ]:
requests.post('http://localhost:9000')

In [ ]:
requests.get('http://localhost:9000')

In [ ]:
requests.put('http://localhost:9000')

In [ ]:
requests.get('http://localhost:9000')

In [ ]:
requests.post('http://localhost:9000')
requests.post('http://localhost:9000')

In [ ]:
requests.get('http://localhost:9000')

In [ ]:
requests.delete('http://localhost:9000')

## Improvement

TODO: response info mimetype, unit tests inline, KERNEL_GATEWAY env var